In [1]:
import numpy as np
import pyspark_helper.util as util
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, coalesce, col, lit, when

In [2]:
spark = SparkSession\
        .builder\
        .getOrCreate()

df = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("train.csv")

# cheeky unpacking to turn column titles to lower
df = df.toDF(*[c.lower() for c in df.columns])

Examine the volume of missing data

In [3]:
missing = util.print_missing(df=df)

Total samples: 891
Column: age, num missing: 177 (% missing: 19.9%)
Column: cabin, num missing: 687 (% missing: 77.1%)
Column: embarked, num missing: 2 (% missing: 0.2%)


Over 3/4 of Cabin data is missing (687/891). There are a total 148 unique values, so 204 of the non-missing samples are unique. There may be useful information here still (e.g. cabin area. i.e. treating A23 and A12 as equivalent). But for now, Cabin is removed.

Embarked has a small amount missing. These are set these to 0 for the sake of one-hot encoding, but will have minimal effect on data. For age, a recursive tree model will be used to predict the remaining values (alternatively, simple mean could be used).

In [4]:
# Drop columns that seem to offer little in terms of predictive value
df = df.drop("passengerid", "name", "ticket", "cabin")
# Create a list of features
initial_features = df.columns
target = "survived"
initial_features.remove(target)

Columns are kept where there is some a priori reason to believe they would act as strong predictors. These include:
Pclass, Age, Sex, SibSp, Parch, Fare, Embarked

Name, PassengerId, and Ticket are omitted as well due to being largely unique values without much discernible useful information

Replace missing values with 0. Since these variables are categorical and 0 is not a natural value, missing data will in essence be its own category.

In [5]:
# all string columns will be required to be encoded
feature_cols_dtypes = df.dtypes
feature_cols_dtypes.pop(0)
features_for_encoding = [col for col, _ in feature_cols_dtypes if _ == "string"]
numeric_features = [col for col, _ in feature_cols_dtypes if _ != "string"]
print("String features for encoding: {0}\
\nNumeric features: {1}".format(features_for_encoding, numeric_features))
df = df.na.fill("0", subset=features_for_encoding)
print("-"*8, "Columns with null values printed below", "-"*8)
missing = util.print_missing(df)

String features for encoding: ['sex', 'embarked']
Numeric features: ['pclass', 'age', 'sibsp', 'parch', 'fare']
-------- Columns with null values printed below --------
Total samples: 891
Column: age, num missing: 177 (% missing: 19.9%)


The one-hot transformer for Sex, Cabin, and Embarked is created and kept for use later on the test set (or during cross-validation).
Pipeline: String -> StringIndex -> OneHotEncoding

In [6]:
onehot_transformer = util.string_to_onehot_transformer(df=df, columns_for_encoding=features_for_encoding)
df = onehot_transformer.transform(df)

As can be seen above, age has many missing values. Age is numerical but holds important information, so it is preferable not to drop it. The missing values can be imputed with the mean, but instead will be predicted using a simple tree model. Since this is only to fill missing data, there will not be extensive testing/cross-validating/ensembling (random forest/boosting).

Below, the predictors for age are vectorised. Then, the decision tree regressor is trained.

In [7]:
feature_cols = numeric_features + [col+"_encoded" for col in features_for_encoding]
age_predictors = feature_cols.copy()
age_predictors.remove("age")
vectoriser = util.create_feature_vectoriser(df, age_predictors, "age_predictors")
df = vectoriser.transform(df)

In [8]:
dt = DecisionTreeRegressor(featuresCol="age_predictors", \
                           labelCol="age", \
                           predictionCol="age_prediction", \
                           maxDepth=5)
df_filtered = df.filter(df["age"].isNotNull())
dt_model = dt.fit(df_filtered)
df = dt_model.transform(df)
# df_filtered is recreated to now incorporate the prediction column for evaluation
df_filtered = df.filter(df["age"].isNotNull())

Now that the age has been predicted, its effectiveness against a simple mean impute is evaluated. RMSE is a quick metric for comparison.

In [9]:
mean_age = df_filtered.agg(avg(col("age"))).collect()[0][0]
print("Mean of non-null values in training set: {0:.2f}".format(mean_age))
df_filtered = df_filtered.withColumn("mean_age", lit(mean_age))
evaluator = RegressionEvaluator()\
        .setMetricName("rmse")\
        .setLabelCol("age")\
        .setPredictionCol("age_prediction")
prediction_rmse = evaluator.evaluate(df_filtered)
evaluator = evaluator.setPredictionCol("mean_age")
mean_rmse = evaluator.evaluate(df_filtered)
print("RMSE of predictions against training set: {0:.2f}\
        \nRMSE of mean value against training set: {1:.2f}".format(prediction_rmse, mean_rmse))

Mean of non-null values in training set: 29.70
RMSE of predictions against training set: 11.27        
RMSE of mean value against training set: 14.52


While not mindblowing by any means, the decision tree has reduced RMSE quite significantly from 14.52. 

Now all features are clean from null values. The original "age" column takes the prediction where "age" is null, and the predictors and prediction columns for age are no longer required. Then, the final features for the survival model are vectorised.

In [10]:
df = df.withColumn("age", coalesce(df.age, df.age_prediction))
df = df.drop("age_predictors", "age_prediction")
feature_vectoriser = util.create_feature_vectoriser(df, feature_cols, "features")
df = feature_vectoriser.transform(df)

In [11]:
print("-"*8, f"Columns to be used as features")
print(feature_cols, "\n")
print("-"*8, "Top 5 rows of label and vectorised features")
df.select("survived", "features").show(5)

-------- Columns to be used as features
['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex_encoded', 'embarked_encoded'] 

-------- Top 5 rows of label and vectorised features
+--------+--------------------+
|survived|            features|
+--------+--------------------+
|       0|[3.0,22.0,1.0,0.0...|
|       1|[1.0,38.0,1.0,0.0...|
|       1|(9,[0,1,4,6],[3.0...|
|       1|[1.0,35.0,1.0,0.0...|
|       0|[3.0,35.0,0.0,0.0...|
+--------+--------------------+
only showing top 5 rows



In [12]:
df.select(*feature_cols).show(5)

+------+----+-----+-----+-------+-------------+----------------+
|pclass| age|sibsp|parch|   fare|  sex_encoded|embarked_encoded|
+------+----+-----+-----+-------+-------------+----------------+
|     3|22.0|    1|    0|   7.25|(1,[0],[1.0])|   (3,[0],[1.0])|
|     1|38.0|    1|    0|71.2833|    (1,[],[])|   (3,[1],[1.0])|
|     3|26.0|    0|    0|  7.925|    (1,[],[])|   (3,[0],[1.0])|
|     1|35.0|    1|    0|   53.1|    (1,[],[])|   (3,[0],[1.0])|
|     3|35.0|    0|    0|   8.05|(1,[0],[1.0])|   (3,[0],[1.0])|
+------+----+-----+-----+-------+-------------+----------------+
only showing top 5 rows



Note AUC will be used as the evaluation metric, so thresholds are not a concern. Model will be first run with the base hyperparameters before performing random (as opposed to grid) search with 5-fold CV.

In [13]:
df = df.withColumnRenamed("survived", "label")

# df = df.drop(*["prediction", "probability", "rawPrediction"]) # uncomment when testing

gbt = GBTClassifier()

gbt_transformer = gbt.fit(df)
df = gbt_transformer.transform(df)

In [14]:
df.select(*["label", "prediction", "probability", "rawPrediction"]).show(5)
print("Incorrect predictions:",df.filter(df.label != df.prediction).count())

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print(evaluator.evaluate(df))

+-----+----------+--------------------+--------------------+
|label|prediction|         probability|       rawPrediction|
+-----+----------+--------------------+--------------------+
|    0|       0.0|[0.91398813051760...|[1.18166614054903...|
|    1|       1.0|[0.05278475052523...|[-1.4436520151715...|
|    1|       1.0|[0.48663789379179...|[-0.0267305771414...|
|    1|       1.0|[0.04441291376077...|[-1.5343978119854...|
|    0|       0.0|[0.88634908728346...|[1.02698964775194...|
+-----+----------+--------------------+--------------------+
only showing top 5 rows

Incorrect predictions: 91
0.9542655972049133


The model has very high AUC. Even with the default threshold of 0.5, it is only inaccurate for 91 rows... But it is obviously overfitting. Time for cross validation and random parameter search. The final parameters will be those which maximise the OOF metric (AUC).

Note that below, the paramGrid is not actually a grid and only offers one option per hyperparameter. This is because the intention is to perform random search rather than grid search. There may be a more elegant solution

A few temporary functions are defined:
get_random_lr returns a random learning rate, reasonably between 0.01 and 1 (using negative log)

In [15]:
def get_random_stepSize():
    rng = np.random.rand() * -2
    return np.power(10, rng)

def get_random_maxDepth(min_depth, max_depth):
    return np.random.randint(min_depth, max_depth+1)

def get_random_maxIter(min_iters, max_iters):
    return np.random.randint(min_iters, max_iters+1)

df = df.drop(*["prediction", "probability", "rawPrediction"])

num_models = 20
models = []
param_maps = []
for i in range(num_models):
    stepSize = get_random_stepSize()
    maxDepth = get_random_maxDepth(3, 8)
    maxIter = get_random_maxIter(10, 30)
    param_maps.append(ParamGridBuilder()\
            .addGrid(gbt.stepSize, [stepSize])\
            .addGrid(gbt.maxDepth, [maxDepth])\
            .addGrid(gbt.maxIter, [maxIter])\
            .build()[0])

crossval = CrossValidator(estimator=gbt,
    estimatorParamMaps=param_maps,
    evaluator=BinaryClassificationEvaluator(),
    numFolds=5)
cvModel = crossval.fit(df)

In [16]:
list(param_maps[0].keys())[0]

Param(parent='GBTClassifier_5f83beb35ae7', name='stepSize', doc='Step size (a.k.a. learning rate) in interval (0, 1] for shrinking the contribution of each estimator.')

In [17]:
bestModel = cvModel.bestModel
bestModel_params = bestModel.extractParamMap()
best_params_dict = {}
for key in param_maps[0].keys():
    best_params_dict[key] = bestModel_params[key]

In [18]:
for k,v in best_params_dict.items():
    print(k, v)

GBTClassifier_5f83beb35ae7__stepSize 0.09811242482588277
GBTClassifier_5f83beb35ae7__maxDepth 5
GBTClassifier_5f83beb35ae7__maxIter 24


In [20]:
print(f"Worst model AUC: {min(cvModel.avgMetrics):.2f}\n\
    Best model AUC: {max(cvModel.avgMetrics):.2f}")

Worst model AUC: 0.84
    Best model AUC: 0.87
